## LangGraph Multi-Agent Mortgage Assistant with Bedrock AgentCore

In [ ]:
%pip install -r requirements.txt -q --force-reinstall

This notebook demonstrates how to deploy a sophisticated multi-agent mortgage assistant using LangGraph and Amazon Bedrock AgentCore.

### Prerequisites
- Knowledge base created (run `01_create_knowledgebase.ipynb` first)
- Python 3.8+ environment

### Setup and Configuration

We'll start by setting up the Bedrock AgentCore starter toolkit

The [Bedrock AgentCore Starter Toolkit](https://github.com/aws/bedrock-agentcore-starter-toolkit/) is a lightweight Python wrapper that streamlines deploying and managing AI agents on AWS Bedrock AgentCore. The toolkit acts as a thin layer over AWS Bedrock AgentCore's control and data plane APIs, automating infrastructure provisioning (ECR repositories, IAM roles, CodeBuild projects) and managing agent state through a .bedrock_agentcore.yaml configuration file. It provides both CLI and notebook interfaces that orchestrate the complete agent lifecycle - from initial configuration and containerization to cloud deployment and invocation. It supports multiple deployment modes (local development, cloud deployment with CodeBuild for ARM64 builds) and handles authentication patterns (JWT, OAuth, IAM) and protocol interfaces (HTTP, Model Context Protocol), enabling developers to focus on building agents rather than wrestling with AWS service integration details.

**Note**: If running locally, uncomment and update the `AWS_PROFILE` line below with your AWS profile name.

In [ ]:
import os
import boto3
import json
import random
import string
from bedrock_agentcore_starter_toolkit import Runtime

# os.environ['AWS_PROFILE']='stcgenai'

session = boto3.Session()
region = session.region_name
print(f"current region: {region}")
account_id = session.client("sts").get_caller_identity()["Account"]
print(f"current account: {account_id}")


### Create IAM Execution Role

The AgentCore runtime requires an IAM role with specific permissions to:
- Access Bedrock models for AI inference
- Retrieve from knowledge bases (KB)
- Write to CloudWatch logs
- Access SSM Parameter for KB ID
- Pull/push Docker images to ECR

We'll create this role using our predefined policy templates that include all necessary permissions.


In [ ]:
# Load policy from external file
with open('policy.json', 'r') as f:
    policy_template = f.read()

# load trust policy from external file
with open('trust-policy.json', 'r') as f:
    trust_policy_template = f.read()

policy = policy_template.replace('REGION', region).replace('ACCOUNT_ID', account_id).replace('REPO_ARN', '*')
print(f"Policy loaded and updated with region: {region}, account: {account_id}")

trust_policy = trust_policy_template.replace('REGION', region).replace('ACCOUNT_ID', account_id)

# create IAM role using the policies
suffix = random.choices(string.ascii_lowercase + string.digits, k=8)
iam_client = boto3.client('iam')
role_name = f"bedrock-runtime-execution-role-{''.join(suffix)}"

role = iam_client.create_role(
    RoleName=role_name,
    AssumeRolePolicyDocument=trust_policy
)

iam_client.put_role_policy(
    RoleName=role_name,
    PolicyName='bedrock-runtime-execution-policy',
    PolicyDocument=policy
)

### Initialize AgentCore Runtime

The Runtime class provides a high-level interface to:
- Configure agent deployment settings
- Build and push Docker containers
- Launch agents on AWS infrastructure
- Invoke deployed agents

This abstracts away the complexity of container orchestration and AWS service integration.


In [21]:
agentcore_runtime = Runtime()

### Configure Agent Deployment

The `configure()` method sets up the deployment infrastructure:


**What happens during configuration:**
- Creates ECR repository and IAM role (if needed)
- Generates Dockerfile and .dockerignore for containerization
- Creates `.bedrock_agentcore.yaml` configuration file

Here is the full parameter list
```
result = configure_bedrock_agentcore(
            agent_name=agent_name,
            entrypoint_path=Path(entrypoint),
            execution_role=execution_role,
            ecr_repository=ecr_repository,
            container_runtime=container_runtime,
            auto_create_ecr=auto_create_ecr,
            enable_observability=not disable_otel,
            requirements_file=final_requirements_file,
            authorizer_configuration=oauth_config,
            verbose=verbose,
            region=region,
            protocol=protocol.upper() if protocol else None,
        )
```

In [ ]:
agent_name = "mortage_assistant"
response = agentcore_runtime.configure(
    entrypoint="langgraph-multi-agent.py", #langgraph-single-agent.py
    execution_role=role['Role']['Arn'],
    # auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name

)

### Launch Agent Runtime

The `launch()` method triggers a CodeBuild job that:

1. **Pre-build**: Authenticates with ECR registry
2. **Build**: Creates ARM64 Docker image with your agent code
3. **Post-build**: Pushes image to ECR and creates a Bedrock AgentCore runtime and deploys the agent


In [ ]:
launch_result = agentcore_runtime.launch()

Lets check the status of the agentcore runtime

In [ ]:
import time
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
print(status)

Now, we we can invoke the runtime agent. The runtime agent will invoke the agent and the agent will invoke the model. The runtime agent will wait for the response from the model and return the response to the user.

In [ ]:
invoke_response = agentcore_runtime.invoke({"prompt": "I am customer 123456 and I am considering refinancing my current mortgage. Before I make a decision, I would like to know my current mortgage details including my interest rate and remaining balance. Could you explain the main benefits and drawbacks of refinancing from a 30-year to a 15-year mortgage, and finally I am also curious about the current 15-year fixed rates to see if my mortgage repayments will be low"})

# Can you help me with mortgage info. Here is my customer id: 23455"
# I am customer 123456 and I am considering refinancing my current mortgage. Before I make a decision, I would like to know my current mortgage details including my interest rate and remaining balance. Could you explain the main benefits and drawbacks of refinancing from a 30-year to a 15-year mortgage, and finally I am also curious about the current 15-year fixed rates to see if my mortgage repayments will be low

### Parse and Display Agent Response

The response contains the full conversation flow between agents, including:
- Agent handoffs and routing decisions
- Tool calls to retrieve mortgage data
- Knowledge base queries for general information
- Final consolidated response to the customer


In [28]:
data = json.loads(b''.join(invoke_response['response']).decode('utf-8'))

messages = data['messages']

In [ ]:
for msg in messages:
    content = msg['content']
    if isinstance(content, list):
        content = ' '.join([item.get('text', str(item)) for item in content if isinstance(item, dict)])
    print(f"\n{msg['type'].upper()}: {content}")

## Summary

You've successfully deployed a multi-agent mortgage assistant that:

✅ **Routes customers** to appropriate specialists automatically  
✅ **Retrieves real-time data** using  tools  
✅ **Accesses knowledge base** for general mortgage guidance  
✅ **Handles complex queries** requiring multiple agent expertise  



## Related Resources

- [Bedrock AgentCore Documentation](https://docs.aws.amazon.com/bedrock/)
- [LangGraph Guide](https://langchain-ai.github.io/langgraph/guides/)
